In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import plotly.graph_objects as go

In [34]:
data = pd.read_csv('data/AVG.csv', delimiter=';')
data.head()

,Players,Ratings,Minutes,Goals,Assists,Offsides,Passes,Accurate_Passes,Accurate_Passes_%,Crosses,Tackles,Total_Shots,Shots_on_Target,Blocked_Shots,Yellow_Cards,Red_Cards,Fouls_Drawn,Fouls_Committed,Clearances,Saves
0,D. Abramowicz,"7,53","90,00","0,33","0,00","0,67","24,67","19,00","75,35","2,33","0,33","1,33","0,33","0,00","0,00","0,00","0,67","0,33","7,67","0,00"
1,S. Kerk,"6,91","76,33","0,00","0,33","0,00","23,67","17,33","70,40","5,67","0,67","2,00","0,67","0,67","0,00","0,00","0,67","0,67","1,67","0,00"
2,J. Oliveira,"6,90","44,33","0,00","0,00","0,67","10,00","8,00","80,00","1,33","0,33","1,00","0,67","0,00","0,00","0,00","1,33","0,00","0,00","0,00"
3,A. Nguiamba,"4,49","29,67","0,00","0,00","0,00","11,00","9,33","56,17","0,00","1,33","0,00","0,00","0,00","0,00","0,00","0,33","0,33","0,67","0,00"
4,D. Zator,"4,53","22,00","0,00","0,00","0,00","12,33","9,33","49,47","1,00","1,33","0,00","0,00","0,00","0,00","0,00","0,33","0,00","1,00","0,00"


In [35]:
data = data.set_index('Players')
data.head()

,Ratings,Minutes,Goals,Assists,Offsides,Passes,Accurate_Passes,Accurate_Passes_%,Crosses,Tackles,Total_Shots,Shots_on_Target,Blocked_Shots,Yellow_Cards,Red_Cards,Fouls_Drawn,Fouls_Committed,Clearances,Saves
Players,,,,,,,,,,,,,,,,,,,
D. Abramowicz,"7,53","90,00","0,33","0,00","0,67","24,67","19,00","75,35","2,33","0,33","1,33","0,33","0,00","0,00","0,00","0,67","0,33","7,67","0,00"
S. Kerk,"6,91","76,33","0,00","0,33","0,00","23,67","17,33","70,40","5,67","0,67","2,00","0,67","0,67","0,00","0,00","0,67","0,67","1,67","0,00"
J. Oliveira,"6,90","44,33","0,00","0,00","0,67","10,00","8,00","80,00","1,33","0,33","1,00","0,67","0,00","0,00","0,00","1,33","0,00","0,00","0,00"
A. Nguiamba,"4,49","29,67","0,00","0,00","0,00","11,00","9,33","56,17","0,00","1,33","0,00","0,00","0,00","0,00","0,00","0,33","0,33","0,67","0,00"
D. Zator,"4,53","22,00","0,00","0,00","0,00","12,33","9,33","49,47","1,00","1,33","0,00","0,00","0,00","0,00","0,00","0,33","0,00","1,00","0,00"


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, D. Abramowicz to E.Espiau
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Ratings            19 non-null     object
 1   Minutes            19 non-null     object
 2   Goals              19 non-null     object
 3   Assists            19 non-null     object
 4   Offsides           19 non-null     object
 5   Passes             19 non-null     object
 6   Accurate_Passes    19 non-null     object
 7   Accurate_Passes_%  19 non-null     object
 8   Crosses            19 non-null     object
 9   Tackles            19 non-null     object
 10  Total_Shots        19 non-null     object
 11  Shots_on_Target    19 non-null     object
 12  Blocked_Shots      19 non-null     object
 13  Yellow_Cards       19 non-null     object
 14  Red_Cards          19 non-null     object
 15  Fouls_Drawn        19 non-null     object
 16  Fouls_Committed    19 non-null   

In [37]:
data = data.replace(',', '.', regex=True)
data = data.apply(pd.to_numeric, errors='coerce')

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, D. Abramowicz to E.Espiau
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ratings            19 non-null     float64
 1   Minutes            19 non-null     float64
 2   Goals              19 non-null     float64
 3   Assists            19 non-null     float64
 4   Offsides           19 non-null     float64
 5   Passes             19 non-null     float64
 6   Accurate_Passes    19 non-null     float64
 7   Accurate_Passes_%  19 non-null     float64
 8   Crosses            19 non-null     float64
 9   Tackles            19 non-null     float64
 10  Total_Shots        19 non-null     float64
 11  Shots_on_Target    19 non-null     float64
 12  Blocked_Shots      19 non-null     float64
 13  Yellow_Cards       19 non-null     float64
 14  Red_Cards          19 non-null     float64
 15  Fouls_Drawn        19 non-null     float64
 16  Fouls_Committed

In [43]:
# Create the figure
from torch import layout


fig = go.Figure()

# List of metrics to plot
metrics = ['Ratings', 'Minutes', 'Goals', 'Assists', 'Offsides', 'Passes',
       'Accurate_Passes', 'Accurate_Passes_%', 'Crosses', 'Tackles',
       'Total_Shots', 'Shots_on_Target', 'Blocked_Shots', 'Yellow_Cards',
       'Red_Cards', 'Fouls_Drawn', 'Fouls_Committed', 'Clearances', 'Saves']

# Add traces for each metric
for metric in metrics:
    filtered_data = data[data[metrics] >  0]
    sorted_df = filtered_data.sort_values(by=metric, ascending=False)
    
    fig.add_trace(go.Bar(x=sorted_df.index, y=sorted_df[metric], name=metric))


# Define visibility matrix
n = len(metrics)
visibility_matrix = {
    "None": [False] * n,
    "AVG Rating": [metric in ['Ratings'] for metric in metrics],
    "Minutes": [metric in ['Minutes'] for metric in metrics],
    "Passes": [metric in ['Passes', 'Accurate_Passes', 'Accurate_Passes_%', 'Crosses'] for metric in metrics],
    "Attack": [metric in ['Goals', 'Assists', 'Total_Shots', 'Shots_on_Target', 'Shots_Inside_Box', 'Shots_Outside_Box'] for metric in metrics],
    "Defense": [metric in ['Tackles', 'Fouls_Drawn', 'Fouls_Committed', 'Saves'] for metric in metrics],
    "Cards": [metric in ['Yellow_Cards', 'Red_Cards'] for metric in metrics],
    "All": [True] * n
}

# Create buttons
buttons = [
    dict(label=label,
         method="update",
         args=[{"visible": visibility},
               {"title": f"Arka Match Stats: {label}"}])
    for label, visibility in visibility_matrix.items()
]

# Add dropdown to figure
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.1,
        y=1.15
    )],
    title="Arka Match Stats",
    xaxis_title="Match",
    yaxis_title="Value",
    barmode='group'
)

fig.show()
fig.write_html("arka_stats_players.html")